# Deploy a Model from a shared Model Package Group

In [ ]:
import json
from sagemaker import ModelPackage
import sagemaker
from sagemaker import get_execution_role
from IPython.core.display import Image, display
from time import gmtime, strftime


Before you get started, check if there are any pending invitations from the shared services account 
and accept them. 
This will allow you to discover share model package groups and register your model versions against them.

In [ ]:
# set required parameters used in cells below
# Hub / Shared Account ID
hub_account_id = 'AWS_ACCOUNT_ID'
# Region used in the Hub account deployments
hub_region = 'AWS_REGION'
# KMS Key id deployed in the Hub account
hub_kms_key_id = 'KMS_KEY_ID'
# S3 bucket name created in the Hub account (The template HubS3BucketName parameter value)
hub_s3_bucket_name = 'HubS3BucketName parameter value used in Cloudformation'

# derive KMS key ARN
hub_kms_key_arn = 'arn:aws:kms:{hub_region}:{hub_account_id}:key/{hub_kms_key_id}'.format(
    hub_account_id=hub_account_id,
    hub_region=hub_region,
    hub_kms_key_id=hub_kms_key_id
)

# set the s3 bucket full name
hub_s3_bucket = 'sagemaker-{hub_region}-{hub_account_id}-{hub_s3_bucket_name}'.format(
    hub_region=hub_region,
    hub_account_id=hub_account_id,
    hub_s3_bucket_name=hub_s3_bucket_name
)

In [ ]:
shared_model_bucket_name = hub_s3_bucket
kms_key_id = hub_kms_key_arn

from botocore.client import ClientError
import boto3
s3_client = boto3.resource('s3')
bucket = s3_client.Bucket(shared_model_bucket_name)
try:
    s3_client.meta.client.head_bucket(Bucket=shared_model_bucket_name)
    print("The bucket "+shared_model_bucket_name+" exists and you have access.")
except ClientError:
    print("The bucket "+shared_model_bucket_name+" does not exist or you have no access.")
    raise

kms_client = boto3.client('kms')
try:
    response = kms_client.describe_key(
        # An identifier for the KMS key. You can use the key ID, key ARN, alias name, alias ARN of the KMS key.
        KeyId=kms_key_id,
    )
    print("The KMS "+kms_key_id+" key exists and you have access.")
except ClientError:
    print("The KMS key "+kms_key_id+" does not exist or you have no access.")
    raise

In [ ]:
role = get_execution_role()
sagemaker_session = sagemaker.Session()
boto3 = sagemaker_session.boto_session
bucket = shared_model_bucket_name
region = sagemaker_session.boto_region_name

s3 = boto3.client("s3")
runtime = boto3.client("runtime.sagemaker")
sagemaker_client = boto3.client("sagemaker")

In [ ]:
response = sagemaker_client.list_model_package_groups(CrossAccountFilterOption="CrossAccount")
print(response)
model_package_group_arn = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupArn']
print(model_package_group_arn)
model_package_group_name = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
print(model_package_group_name)

In [ ]:
real_time_inference_instance_type = "ml.m5.xlarge"

In [ ]:
response = sagemaker_client.list_model_package_groups(CrossAccountFilterOption="CrossAccount")
model_package_group_name = response['ModelPackageGroupSummaryList'][0]['ModelPackageGroupName']
print("Found Model Package Group with name",model_package_group_name)

response = sagemaker_client.list_model_packages(
    # NameContains='string',
    ModelApprovalStatus='Approved',
    ModelPackageGroupName=model_package_group_arn,
    # ModelPackageType='Versioned',
    SortBy='CreationTime',
    SortOrder='Descending'
)
# print(model_package_group_name,response)

if len(response['ModelPackageSummaryList'])>0:
    model_version_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
else:
    print("Could not find a model in Approved state")
    raise

print("Selected latest created and approved model, with model_version_arn:",model_version_arn)

In [ ]:
container_list = [{'ModelPackageName': model_version_arn}]
container_list

#### Before to be able to create a model, you need to move the model status to "Approved" in the Shared Registry

In [ ]:
model_name = "CreditRisk"

In [ ]:
container_list = [
    {
        'ModelPackageName': model_version_arn
    }
]

create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = container_list,
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

In [ ]:
endpoint_config_name = 'DEMO-CreditRisk-EndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    KmsKeyId=kms_key_id,  # This is the correct place to specify the KMS key  
    DataCaptureConfig={
        'EnableCapture': True,
        'InitialSamplingPercentage': 100,
        'DestinationS3Uri': 's3://' + hub_s3_bucket + '/model-monitor/',
        'KmsKeyId': kms_key_id,
        'CaptureOptions': [
            {
                # 'CaptureMode': 'InputAndOutput' # endpoint create fails with this mode
                'CaptureMode': 'Input'
            },
        ],
        'CaptureContentTypeHeader': {
            'CsvContentTypes': [
                'text/csv',
            ],
            'JsonContentTypes': [
                'application/json',
            ]
        }
    },
    ProductionVariants=[{
        'InstanceType':real_time_inference_instance_type,
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}],
        Tags=[
            {
                'Key': 'MODEL_PACKAGE_ARN',
                'Value': model_version_arn
            }
        ])

### Deploy and test the model to an endpoint

In [ ]:
endpoint_name = 'DEMO-CreditRisk-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

# wait for the endpoint to become available
waiter = sagemaker_client.get_waiter('endpoint_in_service')
waiter.wait(
    EndpointName=endpoint_name,
    WaiterConfig={
        'Delay': 30,
        'MaxAttempts': 120
    }
)
print("Endpoint created")

In [ ]:
# Retrieve the last serialized 
s3 = boto3.client('s3')
get_last_modified = lambda obj: int(obj['LastModified'].strftime('%s'))
objs = s3.list_objects_v2(Bucket=hub_s3_bucket)['Contents']
latest_model_featurizer = [obj['Key'] for obj in sorted(objs, key=get_last_modified) if "model_featurizer_" in obj['Key']][-1]
print("Downloading and extracting the latest_model_featurizer",f"s3://{hub_s3_bucket}/{latest_model_featurizer}")


s3 = boto3.client('s3')
s3.download_file(hub_s3_bucket, latest_model_featurizer, 'model_featurizer.tar.gz')
! tar -xvzf model_featurizer.tar.gz

#### UCI Machine Learning Repository Data Usage Disclaimer

Before proceeding with any code execution or data download, please read and acknowledge the following:

#### Disclaimer

The following code and any datasets it may download or use adhere to the UCI Machine Learning Repository citation policy. This includes properly citing both the UCI Machine Learning Repository itself and any relevant papers associated with specific datasets. No modification or distribution of UCI datasets is permitted without proper authorization.

#### Confirmation

Please confirm that you have read and agree to these terms before proceeding

If agree continue to with copying url https://archive.ics.uci.edu/static/public/573/south+german+credit+update.zip and replace in the cell below.

Note: This step reuses the data used to train the original model. We make use of the same data here to illustrate running inferences against an endpoint. You will need to adjust this step to use your data in your environments.

In [ ]:
# Recreate the test.csv dataset locally, using the same sampling seed/state
# download and extract the dataset
!mkdir -p data
!rm -rf data/*
!wget -N --no-check-certificate #replace-url-here
!unzip south+german+credit+update.zip -d data

In [ ]:
import pandas as pd
credit_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
    "credit_risk",
]

training_data = pd.read_csv(
    "data/SouthGermanCredit.asc",
    names=credit_columns,
    header=0,
    sep=r" ",
    engine="python",
    na_values="?",
).dropna()

test_data = training_data.sample(frac=0.1, random_state=42)
test_data = test_data.drop("credit_risk", axis=1)
test_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
]

training_data.to_csv("train.csv", index=False, header=True, columns=credit_columns)
test_data.to_csv("test.csv", index=False, header=True, columns=test_columns)

In [ ]:
# sample the full payload
payload_df = pd.read_csv("test.csv")

realtime_inference_test = payload_df.sample(n=10)

realtime_inference_test.to_csv("realtime.csv", index=False
                               # , header=False
                              )

In [ ]:
# retrieve the samples data
df1 = pd.read_csv('realtime.csv')
# Convert to CSV string
csv_data = df1.to_csv(index=False, header=False)

In [ ]:
import warnings
import pandas as pd
import numpy as np
import tarfile
import sklearn
import joblib
import mlflow
from sagemaker.s3 import S3Uploader
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
from sagemaker.remote_function import remote


def preprocess(df):
    """
    Preprocess the input data and split it into training and validation sets.

    Args:
        df (pandas.DataFrame): Input data.
        experiment_name (str): Name of the MLflow experiment.
        run_id (str, optional): MLflow run ID. If not provided, a new run will be created.
        mlflow_arn (str, optional): MLflow tracking URI.
        s3_root_folder (str, optional): S3 root folder for remote execution.

    Returns:
        tuple: A tuple containing the training and validation features and labels.
    """
    try:
        print("Performing one-hot encoding")
        categorical_cols = [
            "credit_history",
            "purpose",
            "personal_status_sex",
            "other_debtors",
            "property",
            "other_installment_plans",
            "housing",
            "job",
            "telephone",
            "foreign_worker",
        ]
        print("Preparing features and labels")
        X = df.drop("credit_risk", axis=1,errors='ignore')

        with (open("model.joblib", "rb")) as openfile:
            featurizer_model = joblib.load(openfile)
            
        print("Retrieving the scikit-learn transformer",type(featurizer_model))
        X_test = featurizer_model.transform(X)
        print(f"Train features shape after preprocessing: {X_test.shape}")
        
        return X_test
        
    except Exception as e:
        print(f"Exception in processing script: {e}")
        raise e

In [ ]:
paylod_input = preprocess(df1)
print("Number of samples in the payload:",len(paylod_input))

In [ ]:
import boto3
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

# Create predictor
predictor = Predictor(
    endpoint_name=endpoint_name,
    serializer=CSVSerializer()
)

response = predictor.predict(paylod_input,initial_args={'ContentType': 'text/csv'})
print(response)


## uncomment the code below to run a continous loop to run inference for n number of steps
import time
wait_loop_seconds = 60
next_token = time.time() + wait_loop_seconds
n_steps = 1000 # only invoke the endpoint up to this number
t_steps = 0 # keep track of the number of invocations
while True:
    if time.time() > next_token:
        response = predictor.predict(paylod_input,initial_args={'ContentType': 'text/csv'})
        next_token = time.time() + wait_loop_seconds
        t_steps += 1
        if t_steps > n_steps: # requested number of invocations completed, exit loop
            break

## Resources Cleanup

In [ ]:
break # prevent running cells below automatically when running all cells at once

In [ ]:
sagemaker_client.delete_model(
    ModelName = model_name,
)

In [ ]:
sagemaker_client.delete_endpoint_config(
    EndpointConfigName = endpoint_config_name
)

In [ ]:
sagemaker_client.delete_endpoint(
    EndpointName=endpoint_name
)